# PySpark H.W Session 1

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

In [1]:
import spark
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import count
from pyspark.sql.types import *
from pyspark.sql.functions import col
import pyspark.sql.functions as fn



#### Start a simple Spark Session

In [2]:
spark = (SparkSession
.builder
.appName("DataFrame")
.getOrCreate())

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/03 15:23:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sc = spark.sparkContext

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [4]:
walmart_df = spark.read.csv('walmart_stock.csv',header = True,inferSchema=True)

#### What are the column names?

In [5]:
walmart_df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [6]:
walmart_df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### Print out the first 5 columns.

In [7]:
#walmart_df.select('Date', 'Open', 'High', 'Low', 'Close').show()
for line in walmart_df.head(5):
    print(line, '\n')

Row(Date='2012-01-03', Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996) 

Row(Date='2012-01-04', Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475) 

Row(Date='2012-01-05', Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539) 

Row(Date='2012-01-06', Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922) 

Row(Date='2012-01-09', Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004) 



#### Use describe() to learn about the DataFrame.

In [8]:
walmart_df.describe()

DataFrame[summary: string, Date: string, Open: string, High: string, Low: string, Close: string, Volume: string, Adj Close: string]

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [9]:
from pyspark.sql.functions import format_number
summary = walmart_df.describe()
summary.select(summary['summary'],
                  format_number(summary['Open'].cast('float'), 2).alias('Open'),
                  format_number(summary['High'].cast('float'), 2).alias('High'),
                  format_number(summary['Low'].cast('float'), 2).alias('Low'),
                  format_number(summary['Close'].cast('float'), 2).alias('Close'),
                  format_number(summary['Volume'].cast('int'),0).alias('Volume')
                 ).show()

+-------+--------+--------+--------+--------+----------+
|summary|    Open|    High|     Low|   Close|    Volume|
+-------+--------+--------+--------+--------+----------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|     1,258|
|   mean|   72.36|   72.84|   71.92|   72.39| 8,222,093|
| stddev|    6.77|    6.77|    6.74|    6.76| 4,519,780|
|    min|   56.39|   57.06|   56.30|   56.42| 2,094,900|
|    max|   90.80|   90.97|   89.25|   90.47|80,898,100|
+-------+--------+--------+--------+--------+----------+



#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [10]:
wal_new = walmart_df.withColumn('HV Ratio',col('High')/col('Volume'))

In [11]:
wal_new.select('HV Ratio').show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



In [28]:
fn.to_timestamp(walmart_df['Date'])


Column<'to_timestamp(Date)'>

In [30]:
walmart_df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### What day had the Peak High in Price?

In [12]:
walmart_df.sort(walmart_df['High'].desc()).select(['Date']).show(1)  

+----------+
|      Date|
+----------+
|2015-01-13|
+----------+
only showing top 1 row



In [24]:
walmart_df.orderBy(walmart_df['High'].desc()).select(['Date']).head(1)[0]['Date']

'2015-01-13'

#### What is the mean of the Close column?

In [14]:
walmart_df.select(fn.mean('Close')).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



#### What is the max and min of the Volume column?

In [15]:
walmart_df.select(fn.max('Volume').alias('Max Vol'),fn.min('Volume').alias('Min Vol')).show()

+--------+-------+
| Max Vol|Min Vol|
+--------+-------+
|80898100|2094900|
+--------+-------+



#### How many days was the Close lower than 60 dollars?

In [16]:
walmart_df.filter(col('Close') < 60).count()

81

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [17]:
walmart_df.filter(col('High') > 80).count() * 100/ walmart_df.count()

9.141494435612083

#### What is the Pearson correlation between High and Volume?

In [18]:
walmart_df.corr('High','Volume')

-0.3384326061737161

In [19]:
walmart_df.select(fn.corr(col('High'),col('Volume')).alias('High Volume Corr')).show()

+-------------------+
|   High Volume Corr|
+-------------------+
|-0.3384326061737161|
+-------------------+



#### What is the max and min High per year?

In [20]:
year_df = walmart_df.withColumn('Year',fn.year(col('Date')))

year_df.groupBy('Year').agg(fn.max('High'),fn.min('High')).show()

+----+---------+------------------+
|Year|max(High)|         min(High)|
+----+---------+------------------+
|2015|90.970001|         57.060001|
|2013|81.370003| 68.66999799999999|
|2014|88.089996|         73.099998|
|2012|77.599998|             58.43|
|2016|75.190002|61.490002000000004|
+----+---------+------------------+



#### What is the average Close for each Calendar Month?
In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your
result will have a value for each of these months.

In [21]:
month_df = walmart_df.withColumn('Month',fn.month(col('Date')))

month_df = month_df.groupBy('Month').mean()

month_df = month_df.orderBy('Month')

month_df['Month', 'avg(Close)'].show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+

